In [1]:
import numpy as np
import os
import itertools
import sys
sys.path.append('/home/lijunzhang/multibranch/')
from pathlib import Path
from scipy import stats
from scipy.optimize import linear_sum_assignment
from collections import OrderedDict
from ptflops import get_model_complexity_info
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import DataLoader

from framework.layer_node import Conv2dNode, InputNode
from main.layout import Layout
from main.algorithms import enum_layout_wo_rdt, init_S, coarse_to_fined
from main.auto_models import MTSeqBackbone, MTSeqModel, ComputeBlock
from main.head import ASPPHeadNode
from main.trainer import Trainer

from data.nyuv2_dataloader_adashare import NYU_v2
from data.pixel2pixel_loss import NYUCriterions
from data.pixel2pixel_metrics import NYUMetrics

In [2]:
assert torch.cuda.is_available()

# backbone and data

In [3]:
# backbone
# mobilenet
backbone_type = 'mobilenet'
prototxt = '../models/mobilenetv2.prototxt'
D = coarse_B = 5
mapping = {0:[0,1,2,3,4,5,6], 1:[7,8,9,10,11,12,13,14,15,16,17], 2:[18,19,20,21,22], 
           3:[23,24,25,26,27,28,29,30], 4:[31], 5:[32]}

In [4]:
# data
# NYUv2
data = 'NYUv2'
dataroot = '/mnt/nfs/work1/huiguan/lijunzhang/policymtl/data/NYUv2/'
tasks = ['segment_semantic', 'normal', 'depth_zbuffer']
cls_num = {'segment_semantic': 40, 'normal':3, 'depth_zbuffer': 1}

dataset = NYU_v2(dataroot, 'train', crop_h=321, crop_w=321)
trainDataloader = DataLoader(dataset, 32, shuffle=True)
dataset = NYU_v2(dataroot, 'test', crop_h=224, crop_w=224)
valDataloader = DataLoader(dataset, 32, shuffle=True)

criterionDict = {}
metricDict = {}
for task in tasks:
    print(task, flush=True)
    criterionDict[task] = NYUCriterions(task)
    metricDict[task] = NYUMetrics(task)

input_dim = (3,321,321)
T = len(tasks)

segment_semantic
normal
depth_zbuffer


In [5]:
# ind. weights
ckpt_PATH = '/mnt/nfs/work1/huiguan/lijunzhang/multibranch/checkpoint/'
weight_PATH = ckpt_PATH + 'NYUv2/ind/mobilenet/segment_semantic_normal_depth_zbuffer.model' # NYUv2 + MobileNetV2, from the same init
# weight_PATH = ckpt_PATH + 'NYUv2/baseline/WPreMobile/2/segment_semantic_normal_depth_zbuffer.model' # NYUv2 + MobileNetV2, from the same init

# load independent model weights

In [6]:
with torch.no_grad():
    backbone = MTSeqBackbone(prototxt)
    fined_B = len(backbone.basic_blocks)
    feature_dim = backbone(torch.rand(1,3,224,224)).shape[1]

In [7]:
# ind. layout
S = []
for i in range(fined_B):
    S.append([set([x]) for x in range(T)])
layout = Layout(T, fined_B, S) 
print('Ind. Layout:', flush=True)
print(layout, flush=True)

# model
with torch.no_grad():
    model = MTSeqModel(prototxt, layout=layout, feature_dim=feature_dim, cls_num=cls_num)
#     model = model.cuda()

    # load ind. model weights
    model.load_state_dict(torch.load(weight_PATH)['state_dict'])

Ind. Layout:
[[{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}]]
Construct MTSeqModel from Layout:
[[{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}], [{0}, {1}, {2}],

# channel align functions

In [8]:
def simple_alignment(model, tasks, conv_num=-1, verbose=False):
    # Step 1: store tasks' conv modules 
    T = len(tasks)
    task_anchor = tasks[0]
    
    taskConvList = {task: [] for task in tasks}
    count = 0
    for name, module in model.named_modules():
        if count >= conv_num * T and conv_num != -1:
            break
        if isinstance(module, ComputeBlock):
            if len(module.task_set) == 1:
                task_idx = list(module.task_set)[0]
            else:
                print('Wrong independent models with merged branching blocks.')
                sys.exit()
        elif isinstance(module, Conv2dNode):
            taskConvList[tasks[task_idx]].append(module)
            count += 1
    
    # Step 2: align conv weights to task_anchor's in channels
    pre_dist = []
    post_dist = []
    for task in tasks:
        if task == task_anchor:
            continue

        for taskConv in zip(taskConvList[task_anchor],taskConvList[task]):
            # extract weights before alignment
            chn_weights = []
            for i in range(2):
                op = taskConv[i].basicOp
                temp = op.weight.view(op.out_channels, -1) #(out_channels, in_channels*kernel_size), no alignment from prev conv
                chn_weights.append(temp.detach().numpy())

            # compute distance
            dist1 = np.sum(np.linalg.norm(chn_weights[0] - chn_weights[1], axis=1))
            pre_dist.append(dist1)

            # channel alignment
            cost_mat = np.sum(np.power(chn_weights[0], 2),axis=1, keepdims=True) + \
                       np.sum(np.power(chn_weights[1].T, 2),axis=0, keepdims=True) - 2 * (chn_weights[0]@chn_weights[1].T)
            row_ind, col_ind = linear_sum_assignment(cost_mat)

            # save col_ind to ConvNode.out_ord of the second task
            taskConv[1].out_ord = col_ind

            # compute distance
            dist2 = np.sum(np.linalg.norm(chn_weights[0] - chn_weights[1][col_ind,:], axis=1))
            post_dist.append(dist2)
    # print
    if verbose:
        print('pre dist:{}'.format(mean(pre_dist)))
        print('post dist:{}'.format(mean(post_dist)))        
    return

In [9]:
def complex_alignment(model, tasks, conv_num=-1, verbose=False):
    def type_in_list(typ, lst):
        for val in lst:
            if isinstance(val, typ):
                return True
        return False

    def argmin(lst):
        return min(range(len(lst)), key=lst.__getitem__)

    def argmax(lst):
        return max(range(len(lst)), key=lst.__getitem__)

    def father_conv(model, module): 
        fathers = module.fatherNodeList
        if type_in_list(InputNode, fathers):
            # except the first node
            return False
        else:
            # find the father nodes containing Conv2dNode recursively
            while not type_in_list(Conv2dNode, fathers):
                fidx = argmax([x.nodeIdx for x in fathers])
                fathers = fathers[fidx].fatherNodeList
            for f in fathers:
                if isinstance(f, Conv2dNode):
                    return f

    # Step 1: store tasks' conv modules
    T = len(tasks)
    task_anchor = tasks[0]
    
    taskConvList = {task: [] for task in tasks}
    count = 0
    for name, module in model.named_modules():
        if count >= conv_num * T and conv_num != -1:
            break
        if isinstance(module, ComputeBlock):
            if len(module.task_set) == 1:
                task_idx = list(module.task_set)[0]
            else:
                print('Wrong independent models with merged branching blocks.')
                sys.exit()
        elif isinstance(module, Conv2dNode):
            taskConvList[tasks[task_idx]].append(module)
            count += 1

    # Step 2: align conv weights to task_anchor's in channels
    pre_dist = []
    post_dist = []
    for task in tasks:
        if task == task_anchor:
            continue

        for taskConv in zip(taskConvList[task_anchor],taskConvList[task]):
            f = father_conv(model,taskConv[1])
            if f:
                # Corner Case: for GroupConv, set out_ord = father.out_ord, then stop
                if taskConv[1].basicOp.groups > 1:
                    taskConv[1].out_ord = f.out_ord
                    continue
                # Step 2-1: for general Conv, set in_ord=father.out_ord
                else:
                    taskConv[1].in_ord = f.out_ord

            # Step 2-2: extract weights before alignment
            chn_weights = []
            for i in range(2):
                op = taskConv[i].basicOp
                if taskConv[i].in_ord is not None:
                    temp = op.weight[:,taskConv[i].in_ord]
                else:
                    temp = op.weight
                temp = temp.view(op.out_channels, -1) #(out_channels, in_channels*kernel_size)
                chn_weights.append(temp.detach().numpy())

            # compute distance (not good version)
            dist1 = np.sum(np.linalg.norm(chn_weights[0] - \
                      taskConv[1].basicOp.weight.view(op.out_channels, -1).detach().numpy(), axis=1))
            pre_dist.append(dist1)

            # Step 2-3: channel alignment
            cost_mat = np.sum(np.power(chn_weights[0], 2),axis=1, keepdims=True) + \
                       np.sum(np.power(chn_weights[1].T, 2),axis=0, keepdims=True) - 2 * (chn_weights[0]@chn_weights[1].T)
            row_ind, col_ind = linear_sum_assignment(cost_mat)

            # save col_ind to ConvNode.out_ord of the second task
            taskConv[1].out_ord = col_ind

            # compute distance
            dist2 = np.sum(np.linalg.norm(chn_weights[0] - chn_weights[1][col_ind,:], axis=1))
            post_dist.append(dist2)
    # print
    if verbose:
        print('pre dist:{}'.format(mean(pre_dist)))
        print('post dist:{}'.format(mean(post_dist)))
    return

# enum layouts and compute efforts

In [15]:
def dist_effort(weight_list, weight_anchor):
    effort = 0
    for weight in weight_list:
        effort += np.linalg.norm(weight - weight_anchor)
#         effort += np.linalg.norm(weight - weight_anchor) / weight.size # normalization 1.1
#         effort += np.linalg.norm((weight - weight_anchor).flatten(), 1) / weight.size
    return effort

In [16]:
# enum layout
layout_list = [] 
S0 = init_S(T, coarse_B) # initial state
L = Layout(T, coarse_B, S0) # initial layout
layout_list.append(L)
enum_layout_wo_rdt(L, layout_list)

In [17]:
align_choice = 2 # 0: no align; 1: simple align (use out_ord only); 2: complex align

In [18]:
if align_choice == 1:
    simple_alignment(model, tasks)
    
#     for name, module in model.named_modules():
#         if isinstance(module, Conv2dNode):
#             print(name)
#             print(module)
#             print(module.in_ord)
#             print(module.out_ord)
elif align_choice == 2:
    complex_alignment(model, tasks)
    
#     for name, module in model.named_modules():
#         if isinstance(module, Conv2dNode):
#             print(name)
#             print(module)
#             print(module.in_ord)
#             print(module.out_ord)
elif align_choice == 0:
    pass

In [19]:
for L in layout_list:
    layout = coarse_to_fined(L, fined_B, mapping)
    print('Fined Layout:', flush=True)
    print(layout, flush=True)
    
    merge_num = 0
    sum_efforts = 0
    for b in range(layout.B):
        sets = layout.state[b]
        for task_set in sets:
            # if a task set has more than 1 element, merging happens
            if len(task_set) > 1:
                merge_num += len(task_set)-1
                task_convs = [] # store conv weights in each block corresponding to each task

                for task in task_set:
                    # identify task-corresponding block in the well-trained ind. models 
                    temp = []
                    for block in model.backbone.mtl_blocks:
                        if task in block.task_set and block.layer_idx == b:
                            for module in block.compute_nodes:
                                if isinstance(module, Conv2dNode):
                                    temp_weight = module.basicOp.weight.detach().numpy() # no channel alignment or no align variable
                                    
                                    if align_choice == 1 and module.out_ord is not None: # simple alignment
                                        temp_weight = temp_weight[module.out_ord]
                                    elif align_choice == 2: # complex alignment
                                        if module.in_ord is not None:
                                            temp_weight = temp_weight[:,module.in_ord]
                                        if module.out_ord is not None: 
                                            temp_weight = temp_weight[module.out_ord]
                                            
                                    temp.append(temp_weight)
                    task_convs.append(temp)

                # compute effort for each conv
                for conv_idx in range(len(task_convs[0])):
                    conv_weights = []
                    for task_idx in range(len(task_set)):
                        conv_weights.append(task_convs[task_idx][conv_idx])
                    conv_weights = np.array(conv_weights)
                    weight_anchor = np.mean(conv_weights, axis=0)
                    sum_efforts += dist_effort(conv_weights, weight_anchor)
    
    if merge_num == 0:
        L.effort = 0
    else:
        L.effort = sum_efforts/merge_num
#         L.effort = sum_efforts
    print('Effort: {:7f}'.format(L.effort))
    print('-'*100)
#     break

Fined Layout:
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}]]
Effort: 5.389042
----------------------------------------------------------------------------------------------------
Fined Layout:
[[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [

Effort: 2.572655
----------------------------------------------------------------------------------------------------
Fined Layout:
[[{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}]]
Effort: 5.701346
----------------------------------------------------------------------------------------------------
Fined Layout:
[[{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [

Effort: 4.484642
----------------------------------------------------------------------------------------------------
Fined Layout:
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}]]
Effort: 4.867472
----------------------------------------------------------------------------------------------------
Fined Layout:
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0, 1}], [{2}, {0,

Effort: 4.316023
----------------------------------------------------------------------------------------------------
Fined Layout:
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}]]
Effort: 4.232750
----------------------------------------------------------------------------------------------------
Fined Layout:
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}],

In [20]:
# sort
layout_order = sorted(range(len(layout_list)), key=lambda k: layout_list[k],reverse=False)

In [21]:
# choose some layouts to verify
step = 1
for i in range(0,len(layout_order),step):
    print(layout_order[i])
    L = layout_list[layout_order[i]]
    print(L)
    print(L.effort)
    print('-' * 100)

2
[[{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]]
0
----------------------------------------------------------------------------------------------------
18
[[{0, 1, 2}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]]
1.9643150738307409
----------------------------------------------------------------------------------------------------
3
[[{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]]
2.020557028906686
----------------------------------------------------------------------------------------------------
8
[[{1}, {0, 2}], [{1}, {2}, {0}], [{1}, {2}, {0}], [{1}, {2}, {0}], [{1}, {2}, {0}]]
2.217193535396031
----------------------------------------------------------------------------------------------------
13
[[{2}, {0, 1}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}], [{2}, {1}, {0}]]
2.5726546900612965
----------------------------------------------------------------------------------------

In [36]:
# top-k under flops
K = -1
flops = 20
count = 0
for i in range(0,len(layout_order)):
    if count >= K and K != -1:
        break
    L = coarse_to_fined(layout_list[layout_order[i]], fined_B, mapping)
    with torch.no_grad():
        model = MTSeqModel(prototxt, layout=L, feature_dim=feature_dim, cls_num=cls_num, verbose=False)
        macs, params = get_model_complexity_info(model, input_dim, as_strings=True,
                                               print_per_layer_stat=False, verbose=False)
    if float(macs[:-5]) <= flops:
        print(layout_order[i])
        print(layout_list[layout_order[i]])
        print(layout_list[layout_order[i]].effort)
        print(macs)
        print('=' * 100)
        
        count += 1

21
[[{0, 1, 2}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}]]
3.416349628253987
19.98 GMac
17
[[{0, 1, 2}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}]]
3.424751016574028
19.93 GMac
30
[[{0, 1, 2}], [{0, 1, 2}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}]]
4.187049573063851
19.75 GMac
33
[[{0, 1, 2}], [{0, 1, 2}], [{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}]]
4.196091324090958
19.8 GMac
32
[[{0, 1, 2}], [{0, 1, 2}], [{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}]]
4.196550605500617
19.97 GMac
40
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1, 2}, {0}], [{1, 2}, {0}]]
4.512693216042085
19.62 GMac
42
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1, 2}, {0}], [{0}, {2}, {1}]]
4.526928205732946
19.67 GMac
41
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0}, {2}, {1}], [{0}, {2}, {1}]]
4.584874414231466
19.84 GMac
38
[[{0, 1, 2}], [{0, 1, 2}], [{2}, {0, 1}], [{2}, {1}, {0}], [{2}, {1}, {0}]]
4.619047907067508
19.97 GMac
35
[[{0, 1, 2}], [{0, 1, 2}], [{1}, {0, 2}], [{1}

# save init weights for selected mtl model

In [85]:
choose = 1
print('Layout Idx:{}'.format(layout_order[choose]))

Layout Idx:1


In [86]:
layout = coarse_to_fined(layout_list[layout_order[choose]], fined_B, mapping)
mtl_model = MTSeqModel(prototxt, layout=layout, feature_dim=feature_dim, cls_num=cls_num)

Construct MTSeqModel from Layout:
[[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}]]


In [22]:
# create weight init state_dict
mtl_init = OrderedDict()
for name, module in mtl_model.named_modules():
    if isinstance(module, ComputeBlock):
        task_set = module.task_set
        layer_idx = module.layer_idx
        if len(task_set) > 1:
            merge_flag = True
        else:
            # Type 1: save the whole block weights from the corresponding ind. model when no merging
            merge_flag = False
            for block in model.backbone.mtl_blocks:
                if task_set == block.task_set and block.layer_idx == layer_idx:
                    for ind_name, param in block.named_parameters():
                        mtl_init['.'.join([name, ind_name])] = param  
                    # for BN running mean and running var
                    for ind_name, param in block.named_buffers():
                        mtl_init['.'.join([name, ind_name])] = param
                        
    # # Type 2: when the current block have merged operators, save mean weights for convs
    elif isinstance(module, Conv2dNode) and merge_flag: 
        task_convs = [] # store conv weights from task's ind. block
        for task in task_set:
            # identify task-corresponding block in the well-trained ind. models 
            for block in model.backbone.mtl_blocks:
                if task in block.task_set and block.layer_idx == layer_idx:
                    task_module = block.compute_nodes[int(name.split('.')[-1])]  
                    temp_weight = task_module.basicOp.weight # no channel alignment or no align variable
                    if align_choice == 1 and task_module.out_ord is not None: # simple alignment
                        temp_weight = temp_weight[task_module.out_ord]
                    elif align_choice == 2: # complex alignment
                        if task_module.in_ord is not None:
                            temp_weight = temp_weight[:,task_module.in_ord]
                        if task_module.out_ord is not None: 
                            temp_weight = temp_weight[task_module.out_ord]
                    task_convs.append(temp_weight)
        weight_anchor = torch.mean(torch.stack(task_convs),dim=0)
        mtl_init[name+'.basicOp.weight'] = weight_anchor
        
    # Type 3: save heads' weights
    elif 'heads' in name and isinstance(module, ASPPHeadNode): 
        ind_head = model.heads[name.split('.')[-1]]
        for ind_name, param in ind_head.named_parameters():
            mtl_init['.'.join([name, ind_name])] = param
        for ind_name, param in ind_head.named_buffers():
            mtl_init['.'.join([name, ind_name])] = param

In [23]:
mtl_model.load_state_dict(mtl_init,strict=False)

_IncompatibleKeys(missing_keys=['backbone.mtl_blocks.0.compute_nodes.1.basicOp.weight', 'backbone.mtl_blocks.0.compute_nodes.1.basicOp.bias', 'backbone.mtl_blocks.0.compute_nodes.1.basicOp.running_mean', 'backbone.mtl_blocks.0.compute_nodes.1.basicOp.running_var', 'backbone.mtl_blocks.1.compute_nodes.1.basicOp.weight', 'backbone.mtl_blocks.1.compute_nodes.1.basicOp.bias', 'backbone.mtl_blocks.1.compute_nodes.1.basicOp.running_mean', 'backbone.mtl_blocks.1.compute_nodes.1.basicOp.running_var', 'backbone.mtl_blocks.2.compute_nodes.1.basicOp.weight', 'backbone.mtl_blocks.2.compute_nodes.1.basicOp.bias', 'backbone.mtl_blocks.2.compute_nodes.1.basicOp.running_mean', 'backbone.mtl_blocks.2.compute_nodes.1.basicOp.running_var', 'backbone.mtl_blocks.3.compute_nodes.1.basicOp.weight', 'backbone.mtl_blocks.3.compute_nodes.1.basicOp.bias', 'backbone.mtl_blocks.3.compute_nodes.1.basicOp.running_mean', 'backbone.mtl_blocks.3.compute_nodes.1.basicOp.running_var', 'backbone.mtl_blocks.4.compute_nodes

In [24]:
# check weight equality to validate success weight init loading
for name, param in mtl_model.named_parameters():
    if name in mtl_init:
        print(name)
        print(torch.equal(param,mtl_init[name]))

backbone.mtl_blocks.0.compute_nodes.0.basicOp.weight
True
backbone.mtl_blocks.1.compute_nodes.0.basicOp.weight
True
backbone.mtl_blocks.2.compute_nodes.0.basicOp.weight
True
backbone.mtl_blocks.3.compute_nodes.0.basicOp.weight
True
backbone.mtl_blocks.4.compute_nodes.0.basicOp.weight
True
backbone.mtl_blocks.5.compute_nodes.0.basicOp.weight
True
backbone.mtl_blocks.6.compute_nodes.0.basicOp.weight
True
backbone.mtl_blocks.6.compute_nodes.3.basicOp.weight
True
backbone.mtl_blocks.6.compute_nodes.6.basicOp.weight
True
backbone.mtl_blocks.7.compute_nodes.0.basicOp.weight
True
backbone.mtl_blocks.7.compute_nodes.1.basicOp.weight
True
backbone.mtl_blocks.7.compute_nodes.1.basicOp.bias
True
backbone.mtl_blocks.8.compute_nodes.0.basicOp.weight
True
backbone.mtl_blocks.8.compute_nodes.1.basicOp.weight
True
backbone.mtl_blocks.8.compute_nodes.1.basicOp.bias
True
backbone.mtl_blocks.9.compute_nodes.0.basicOp.weight
True
backbone.mtl_blocks.9.compute_nodes.1.basicOp.weight
True
backbone.mtl_block

True
heads.depth_zbuffer.fc4.conv3.weight
True
heads.depth_zbuffer.fc4.conv3.bias
True


# train mtl model

In [25]:
projectroot = '/mnt/nfs/work1/huiguan/lijunzhang/multibranch/'
savepath = os.path.join(projectroot, 'checkpoint', data, 'exp', str(layout_order[choose])+'-cmplx/')
writerpath = os.path.join(projectroot, 'writer', data, 'exp', str(layout_order[choose])+'-cmplx/')
Path(savepath).mkdir(parents=True, exist_ok=True)
Path(writerpath).mkdir(parents=True, exist_ok=True)

In [26]:
mtl_model = mtl_model.cuda()
trainer = Trainer(mtl_model, tasks, trainDataloader, valDataloader, criterionDict, metricDict)

In [ ]:
loss_lambda = {task: 1 for task in tasks}
trainer.train(20000, loss_lambda, savepath, writerPath=writerpath)

[Iter 50 Task segm] Train Loss: 1.4652
[Iter 50 Task norm] Train Loss: 0.0665
[Iter 50 Task dept] Train Loss: 0.9445
[Iter 50 Total] Train Loss: 2.4762
[Iter 100 Task segm] Train Loss: 1.4173
[Iter 100 Task norm] Train Loss: 0.0656
[Iter 100 Task dept] Train Loss: 0.8916
[Iter 100 Total] Train Loss: 2.3745
[Iter 150 Task segm] Train Loss: 1.4317
[Iter 150 Task norm] Train Loss: 0.0647
[Iter 150 Task dept] Train Loss: 0.8982
[Iter 150 Total] Train Loss: 2.3946
[Iter 200 Task segm] Train Loss: 1.3272
[Iter 200 Task norm] Train Loss: 0.0636
[Iter 200 Task dept] Train Loss: 0.8923
[Iter 200 Total] Train Loss: 2.2830
[Iter 200 Task segm] Val Loss: 1.7348
{'mIoU': 0.1499, 'Pixel Acc': 0.4622}
[Iter 200 Task norm] Val Loss: 0.0636
{'Angle Mean': 18.3778, 'Angle Median': 16.7739, 'Angle 11.25': 23.1947, 'Angle 22.5': 74.2376, 'Angle 30': 87.5112}
[Iter 200 Task dept] Val Loss: 0.7768
{'abs_err': 0.7613, 'rel_err': 0.3025, 'sigma_1.25': 48.6441, 'sigma_1.25^2': 79.1389, 'sigma_1.25^3': 92.2169}

[Iter 1250 Task norm] Train Loss: 0.0623
[Iter 1250 Task dept] Train Loss: 0.8011
[Iter 1250 Total] Train Loss: 2.1767
[Iter 1300 Task segm] Train Loss: 1.3369
[Iter 1300 Task norm] Train Loss: 0.0621
[Iter 1300 Task dept] Train Loss: 0.8202
[Iter 1300 Total] Train Loss: 2.2192
[Iter 1350 Task segm] Train Loss: 1.2942
[Iter 1350 Task norm] Train Loss: 0.0619
[Iter 1350 Task dept] Train Loss: 0.8169
[Iter 1350 Total] Train Loss: 2.1730
[Iter 1400 Task segm] Train Loss: 1.2856
[Iter 1400 Task norm] Train Loss: 0.0615
[Iter 1400 Task dept] Train Loss: 0.7974
[Iter 1400 Total] Train Loss: 2.1444
[Iter 1400 Task segm] Val Loss: 1.7421
{'mIoU': 0.1671, 'Pixel Acc': 0.451}
[Iter 1400 Task norm] Val Loss: 0.0641
{'Angle Mean': 17.8792, 'Angle Median': 16.1789, 'Angle 11.25': 30.6313, 'Angle 22.5': 72.4828, 'Angle 30': 86.1862}
[Iter 1400 Task dept] Val Loss: 0.7396
{'abs_err': 0.7579, 'rel_err': 0.3465, 'sigma_1.25': 50.7291, 'sigma_1.25^2': 79.0115, 'sigma_1.25^3': 91.2644}
[Iter 1450 Task se

[Iter 2450 Task segm] Train Loss: 1.2257
[Iter 2450 Task norm] Train Loss: 0.0606
[Iter 2450 Task dept] Train Loss: 0.7701
[Iter 2450 Total] Train Loss: 2.0564
[Iter 2500 Task segm] Train Loss: 1.2325
[Iter 2500 Task norm] Train Loss: 0.0610
[Iter 2500 Task dept] Train Loss: 0.7691
[Iter 2500 Total] Train Loss: 2.0627
[Iter 2550 Task segm] Train Loss: 1.2118
[Iter 2550 Task norm] Train Loss: 0.0619
[Iter 2550 Task dept] Train Loss: 0.7804
[Iter 2550 Total] Train Loss: 2.0541
[Iter 2600 Task segm] Train Loss: 1.2118
[Iter 2600 Task norm] Train Loss: 0.0617
[Iter 2600 Task dept] Train Loss: 0.7937
[Iter 2600 Total] Train Loss: 2.0672
[Iter 2600 Task segm] Val Loss: 1.7311
{'mIoU': 0.1675, 'Pixel Acc': 0.4529}
[Iter 2600 Task norm] Val Loss: 0.0623
{'Angle Mean': 18.3316, 'Angle Median': 16.9493, 'Angle 11.25': 23.7704, 'Angle 22.5': 74.6822, 'Angle 30': 87.8216}
[Iter 2600 Task dept] Val Loss: 0.8606
{'abs_err': 0.8357, 'rel_err': 0.2926, 'sigma_1.25': 43.5102, 'sigma_1.25^2': 73.9955, '

[Iter 3650 Task segm] Train Loss: 1.1895
[Iter 3650 Task norm] Train Loss: 0.0613
[Iter 3650 Task dept] Train Loss: 0.7650
[Iter 3650 Total] Train Loss: 2.0158
[Iter 3700 Task segm] Train Loss: 1.1694
[Iter 3700 Task norm] Train Loss: 0.0621
[Iter 3700 Task dept] Train Loss: 0.7724
[Iter 3700 Total] Train Loss: 2.0039
[Iter 3750 Task segm] Train Loss: 1.1921
[Iter 3750 Task norm] Train Loss: 0.0625
[Iter 3750 Task dept] Train Loss: 0.7699
[Iter 3750 Total] Train Loss: 2.0244
[Iter 3800 Task segm] Train Loss: 1.1587
[Iter 3800 Task norm] Train Loss: 0.0611
[Iter 3800 Task dept] Train Loss: 0.7505
[Iter 3800 Total] Train Loss: 1.9703
[Iter 3800 Task segm] Val Loss: 1.6558
{'mIoU': 0.1853, 'Pixel Acc': 0.4744}
[Iter 3800 Task norm] Val Loss: 0.0622
{'Angle Mean': 18.0734, 'Angle Median': 16.6388, 'Angle 11.25': 27.2398, 'Angle 22.5': 74.0533, 'Angle 30': 86.7157}
[Iter 3800 Task dept] Val Loss: 0.7246
{'abs_err': 0.7187, 'rel_err': 0.2803, 'sigma_1.25': 51.577, 'sigma_1.25^2': 81.1516, 's

[Iter 4850 Task segm] Train Loss: 0.9384
[Iter 4850 Task norm] Train Loss: 0.0596
[Iter 4850 Task dept] Train Loss: 0.7000
[Iter 4850 Total] Train Loss: 1.6980
[Iter 4900 Task segm] Train Loss: 0.9616
[Iter 4900 Task norm] Train Loss: 0.0602
[Iter 4900 Task dept] Train Loss: 0.6950
[Iter 4900 Total] Train Loss: 1.7168
[Iter 4950 Task segm] Train Loss: 0.9190
[Iter 4950 Task norm] Train Loss: 0.0589
[Iter 4950 Task dept] Train Loss: 0.7055
[Iter 4950 Total] Train Loss: 1.6834
[Iter 5000 Task segm] Train Loss: 0.9287
[Iter 5000 Task norm] Train Loss: 0.0599
[Iter 5000 Task dept] Train Loss: 0.7012
[Iter 5000 Total] Train Loss: 1.6898
[Iter 5000 Task segm] Val Loss: 1.5818
{'mIoU': 0.2038, 'Pixel Acc': 0.4853}
[Iter 5000 Task norm] Val Loss: 0.0628
{'Angle Mean': 17.9257, 'Angle Median': 16.6432, 'Angle 11.25': 28.6992, 'Angle 22.5': 71.5348, 'Angle 30': 86.4733}
[Iter 5000 Task dept] Val Loss: 0.6618
{'abs_err': 0.6695, 'rel_err': 0.2824, 'sigma_1.25': 55.3207, 'sigma_1.25^2': 83.7711, '

[Iter 6050 Task segm] Train Loss: 0.9100
[Iter 6050 Task norm] Train Loss: 0.0592
[Iter 6050 Task dept] Train Loss: 0.6807
[Iter 6050 Total] Train Loss: 1.6499
[Iter 6100 Task segm] Train Loss: 0.9697
[Iter 6100 Task norm] Train Loss: 0.0592
[Iter 6100 Task dept] Train Loss: 0.6835
[Iter 6100 Total] Train Loss: 1.7124
[Iter 6150 Task segm] Train Loss: 0.9292
[Iter 6150 Task norm] Train Loss: 0.0595
[Iter 6150 Task dept] Train Loss: 0.6760
[Iter 6150 Total] Train Loss: 1.6647
[Iter 6200 Task segm] Train Loss: 0.9288
[Iter 6200 Task norm] Train Loss: 0.0620
[Iter 6200 Task dept] Train Loss: 0.6841
[Iter 6200 Total] Train Loss: 1.6750
[Iter 6200 Task segm] Val Loss: 1.5826
{'mIoU': 0.2042, 'Pixel Acc': 0.4771}
[Iter 6200 Task norm] Val Loss: 0.0635
{'Angle Mean': 18.1322, 'Angle Median': 17.1321, 'Angle 11.25': 28.7171, 'Angle 22.5': 69.6054, 'Angle 30': 86.2562}
[Iter 6200 Task dept] Val Loss: 0.7550
{'abs_err': 0.7335, 'rel_err': 0.2603, 'sigma_1.25': 50.1455, 'sigma_1.25^2': 80.6216, '

[Iter 7250 Task segm] Train Loss: 0.8575
[Iter 7250 Task norm] Train Loss: 0.0587
[Iter 7250 Task dept] Train Loss: 0.6783
[Iter 7250 Total] Train Loss: 1.5946
[Iter 7300 Task segm] Train Loss: 0.8800
[Iter 7300 Task norm] Train Loss: 0.0600
[Iter 7300 Task dept] Train Loss: 0.6696
[Iter 7300 Total] Train Loss: 1.6095
[Iter 7350 Task segm] Train Loss: 0.8636
[Iter 7350 Task norm] Train Loss: 0.0597
[Iter 7350 Task dept] Train Loss: 0.6514
[Iter 7350 Total] Train Loss: 1.5747
[Iter 7400 Task segm] Train Loss: 0.8810
[Iter 7400 Task norm] Train Loss: 0.0600
[Iter 7400 Task dept] Train Loss: 0.6627
[Iter 7400 Total] Train Loss: 1.6037
[Iter 7400 Task segm] Val Loss: 1.5563
{'mIoU': 0.2066, 'Pixel Acc': 0.4944}
[Iter 7400 Task norm] Val Loss: 0.0616
{'Angle Mean': 17.9897, 'Angle Median': 16.3514, 'Angle 11.25': 28.0885, 'Angle 22.5': 72.8922, 'Angle 30': 87.0267}
[Iter 7400 Task dept] Val Loss: 0.6400
{'abs_err': 0.6485, 'rel_err': 0.2741, 'sigma_1.25': 57.2931, 'sigma_1.25^2': 84.6093, '

[Iter 8450 Task segm] Train Loss: 0.7410
[Iter 8450 Task norm] Train Loss: 0.0572
[Iter 8450 Task dept] Train Loss: 0.6494
[Iter 8450 Total] Train Loss: 1.4476
[Iter 8500 Task segm] Train Loss: 0.7755
[Iter 8500 Task norm] Train Loss: 0.0573
[Iter 8500 Task dept] Train Loss: 0.6396
[Iter 8500 Total] Train Loss: 1.4724
[Iter 8550 Task segm] Train Loss: 0.7445
[Iter 8550 Task norm] Train Loss: 0.0582
[Iter 8550 Task dept] Train Loss: 0.6356
[Iter 8550 Total] Train Loss: 1.4383
[Iter 8600 Task segm] Train Loss: 0.7492
[Iter 8600 Task norm] Train Loss: 0.0558
[Iter 8600 Task dept] Train Loss: 0.6145
[Iter 8600 Total] Train Loss: 1.4195
[Iter 8600 Task segm] Val Loss: 1.5413
{'mIoU': 0.2122, 'Pixel Acc': 0.4966}
[Iter 8600 Task norm] Val Loss: 0.0608
{'Angle Mean': 17.6392, 'Angle Median': 15.8632, 'Angle 11.25': 30.7778, 'Angle 22.5': 73.1295, 'Angle 30': 86.6357}
[Iter 8600 Task dept] Val Loss: 0.6468
{'abs_err': 0.6492, 'rel_err': 0.2562, 'sigma_1.25': 57.1908, 'sigma_1.25^2': 84.8224, '

[Iter 9650 Task segm] Train Loss: 0.6968
[Iter 9650 Task norm] Train Loss: 0.0589
[Iter 9650 Task dept] Train Loss: 0.6187
[Iter 9650 Total] Train Loss: 1.3744


In [ ]:
# 18=simple-0.01
======================================================================
[Iter 20000 Task segm] Val Loss: 1.6018
{'mIoU': 0.2144, 'Pixel Acc': 0.5058}
[Iter 20000 Task norm] Val Loss: 0.0643
{'Angle Mean': 18.0106, 'Angle Median': 15.8517, 'Angle 11.25': 30.9475, 'Angle 22.5': 71.4377, 'Angle 30': 85.0245}
[Iter 20000 Task dept] Val Loss: 0.6230
{'abs_err': 0.63, 'rel_err': 0.2449, 'sigma_1.25': 58.6577, 'sigma_1.25^2': 85.8279, 'sigma_1.25^3': 95.6178}
======================================================================

# SROCC with loss

In [43]:
import pickle
def save_obj(obj, name):
    with open('./FastMTL/exp/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('./FastMTL/exp/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [144]:
avg_loss_sum=[]
for L in layout_list:
    layout = coarse_to_fined(L, fined_B, mapping)
    print('Fined Layout:', flush=True)
    print(layout, flush=True)
    
    with torch.no_grad():
        mtl_model = MTSeqModel(prototxt, layout=layout, feature_dim=feature_dim, cls_num=cls_num, verbose=False)
        # create weight init state_dict
        mtl_init = OrderedDict()
        for name, module in mtl_model.named_modules():
            if isinstance(module, ComputeBlock):
                task_set = module.task_set
                layer_idx = module.layer_idx
                if len(task_set) > 1:
                    merge_flag = True
                else:
                    # Type 1: save the whole block weights from the corresponding ind. model when no merging
                    merge_flag = False
                    for block in model.backbone.mtl_blocks:
                        if task_set == block.task_set and block.layer_idx == layer_idx:
                            for ind_name, param in block.named_parameters():
                                mtl_init['.'.join([name, ind_name])] = param  
                            # for BN running mean and running var
                            for ind_name, param in block.named_buffers():
                                mtl_init['.'.join([name, ind_name])] = param

            # # Type 2: when the current block have merged operators, save mean weights for convs
            elif isinstance(module, Conv2dNode) and merge_flag: 
                task_convs = [] # store conv weights from task's ind. block
                for task in task_set:
                    # identify task-corresponding block in the well-trained ind. models 
                    for block in model.backbone.mtl_blocks:
                        if task in block.task_set and block.layer_idx == layer_idx:
                            task_module = block.compute_nodes[int(name.split('.')[-1])]  
                            temp_weight = task_module.basicOp.weight # no channel alignment or no align variable
                            if align_choice == 1 and task_module.out_ord is not None: # simple alignment
                                temp_weight = temp_weight[task_module.out_ord]
                            elif align_choice == 2: # complex alignment
                                if task_module.in_ord is not None:
                                    temp_weight = temp_weight[:,task_module.in_ord]
                                if task_module.out_ord is not None: 
                                    temp_weight = temp_weight[task_module.out_ord]
                            task_convs.append(temp_weight)
                weight_anchor = torch.mean(torch.stack(task_convs),dim=0)
                mtl_init[name+'.basicOp.weight'] = weight_anchor

            # Type 3: save heads' weights
            elif 'heads' in name and isinstance(module, ASPPHeadNode): 
                ind_head = model.heads[name.split('.')[-1]]
                for ind_name, param in ind_head.named_parameters():
                    mtl_init['.'.join([name, ind_name])] = param
                for ind_name, param in ind_head.named_buffers():
                    mtl_init['.'.join([name, ind_name])] = param
        mtl_model.load_state_dict(mtl_init,strict=False)
        print('finish weight loading.', flush=True)
        print('-'*50)

        # compute loss
        mtl_model.eval()
        loss_list = {task:[] for task in tasks}
        for i, data in enumerate(trainDataloader):
            x = data['input'].cuda()
            output = mtl_model(x)

            for task in tasks:
                y = data[task].cuda()
                if task + '_mask' in data:
                    tloss = criterionDict[task](output[task], y, data[task + '_mask'].cuda())
                else:
                    tloss = criterionDict[task](output[task], y)
                loss_list[task].append(tloss.item())

        avg_loss = {}
        for task in tasks:
            avg_loss[task] = np.mean(loss_list[task]) 
        print(avg_loss, flush=True)
        avg_loss_sum.append(sum([avg_loss[task] for task in tasks]))

Fined Layout:
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}]]
finish weight loading.
--------------------------------------------------
{'segment_semantic': 4.943176405770438, 'normal': 0.07551473379135132, 'depth_zbuffer': 0.9622610126222882}
Fined Layout:
[[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}

KeyboardInterrupt: 

In [45]:
# Load from pickle directly
avg_loss_sum = load_obj('real_train_error_'+str(align_choice))

In [23]:
avg_loss_sum

[8.423850526809693,
 1.9821560549736024,
 2.003706440925598,
 1.9813451409339906,
 2.018480246067047,
 1.9802546858787535,
 1.9865136456489563,
 5.79551322221756,
 2.239673171043396,
 2.780404443740845,
 3.1638618111610413,
 3.6611135387420655,
 8.457329704761506,
 3.2954431962966924,
 5.071077561378479,
 5.256081328392029,
 5.072607529163361,
 3.1117977094650264,
 3.1767089104652406,
 3.1295277643203736,
 3.124799964427948,
 3.158599801063538,
 6.924007332324981,
 3.7202820324897767,
 4.0713688135147095,
 4.30603806734085,
 8.607510647773744,
 5.054524195194245,
 5.1910976123809816,
 5.1001525449752805,
 4.989250864982605,
 4.9632596278190615,
 4.960848708152771,
 4.974644494056702,
 7.972000014781952,
 5.004748935699463,
 4.844277243614197,
 8.5051616024971,
 5.227097496986389,
 5.101937117576599,
 5.1352508878707885,
 5.184044935703278,
 5.153459544181823,
 8.3842262840271,
 5.061788718700409,
 8.471359386444092,
 5.055881562232971,
 5.095709278583527,
 5.16780980348587,
 8.33889617

In [62]:
# align_choice = 2, load all
stats.spearmanr([L.effort for L in layout_list],avg_loss_sum)

SpearmanrResult(correlation=0.6951131221719457, pvalue=1.515817176020682e-08)

In [28]:
save_obj(avg_loss_sum, 'real_train_error_'+str(align_choice))